<a href="https://colab.research.google.com/github/sanheensethi/gndcteam/blob/master/PlantDisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Downloading Dataset
!git clone https://github.com/spMohanty/PlantVillage-Dataset

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 163229, done.
remote: Total 163229 (delta 0), reused 0 (delta 0), pack-reused 163229
Receiving objects: 100% (163229/163229), 2.00 GiB | 15.56 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Checking out files: 100% (182401/182401), done.


In [4]:
#copy the data
!cp -R /content/PlantVillage-Dataset/raw /content/

In [5]:
#for removing any folder
!rm -r /content/Test

rm: cannot remove '/content/Test': No such file or directory


In [6]:
#Inserting Libraries
import numpy as np
import random
import shutil
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [7]:
 def TestFileFolderCreate() :   
    randomFolder = random.randint(0,len(root_dir))
    folderName = root_dir[randomFolder]
    print(folderName)
    imageList = listdir(f"{directory_root}/{folderName}")
    randomImageNum = random.randint(0,len(imageList))
    ImageName = imageList[randomImageNum]
    print(ImageName)
    WriteFile(folderName,ImageName)
    src = f"{directory_root}/{folderName}/{ImageName}"
    des = '/content/Test/'
    shutil.move(src,des)

In [8]:
#Initializing Variables
EPOCHS = 25
INIT_LR = 1e-4
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = '/content/raw/color'
width=256
height=256
depth=3

In [9]:
root_dir = listdir(directory_root)
print("Number Of Folders : " + str(len(root_dir)))

Number Of Folders : 38


In [10]:
!mkdir Test

In [11]:
def WriteFile(ClassName,ImageName):
    f = open("TestFile.csv","a+")
    f.write(ClassName+"@#"+ImageName+"\n")

In [12]:
for i in range(0,10):
    print(i)
    TestFileFolderCreate()

0
Potato___Early_blight
88c63d75-4423-4233-9097-728af8a52033___RS_Early.B 6862.JPG
1
Corn_(maize)___healthy
9275afa7-daf0-4052-b092-47d36132e69d___R.S_HL 8189 copy 2.jpg
2
Apple___Black_rot
7e008f42-c268-4ca8-9acc-4682019ba060___JR_FrgE.S 8600.JPG
3
Potato___Early_blight
f5fca019-b28f-4000-9d5e-b0b5fd061304___RS_Early.B 7503.JPG
4
Tomato___Bacterial_spot
67b25260-c72a-4ccc-a99b-1e65efbd861c___GCREC_Bact.Sp 6248.JPG
5
Tomato___Tomato_mosaic_virus
29bbbfcc-76ef-4506-9c1b-1a749646856e___PSU_CG 2184.JPG
6
Pepper,_bell___healthy
8de81983-7026-47e2-a972-854af986bfc0___JR_HL 5879.JPG
7
Tomato___healthy
0de569a8-21c5-4d22-b9c9-ba0ef0f2b614___RS_HL 0314.JPG
8
Tomato___Target_Spot
3021f8a0-e41c-4f6d-9b48-0e71e8b7339d___Com.G_TgS_FL 9943.JPG
9
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
3d85efed-2d4b-4f02-8023-144fa4c4bac5___FAM_L.Blight 1637.JPG


In [13]:
#Viewing Number of Images in Folder
print("[INFO] Loading Data ")
root_dir = listdir(directory_root)
sumP = 0
for directory in root_dir :
    print("[FOLDER] Loading Folder : "+str(directory))
    plant_disease_image_list = listdir(f"{directory_root}/{directory}")
    lenP = len(plant_disease_image_list)
    print("[IMAGES] Images : " + str(lenP))
    sumP += lenP
   
print("Total Images : " + str(sumP))

[INFO] Loading Data 
[FOLDER] Loading Folder : Cherry_(including_sour)___healthy
[IMAGES] Images : 854
[FOLDER] Loading Folder : Squash___Powdery_mildew
[IMAGES] Images : 1835
[FOLDER] Loading Folder : Tomato___Leaf_Mold
[IMAGES] Images : 952
[FOLDER] Loading Folder : Tomato___Bacterial_spot
[IMAGES] Images : 2126
[FOLDER] Loading Folder : Corn_(maize)___healthy
[IMAGES] Images : 1161
[FOLDER] Loading Folder : Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
[IMAGES] Images : 513
[FOLDER] Loading Folder : Apple___healthy
[IMAGES] Images : 1645
[FOLDER] Loading Folder : Grape___Esca_(Black_Measles)
[IMAGES] Images : 1383
[FOLDER] Loading Folder : Apple___Apple_scab
[IMAGES] Images : 630
[FOLDER] Loading Folder : Apple___Black_rot
[IMAGES] Images : 620
[FOLDER] Loading Folder : Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
[IMAGES] Images : 1075
[FOLDER] Loading Folder : Grape___Black_rot
[IMAGES] Images : 1180
[FOLDER] Loading Folder : Pepper,_bell___healthy
[IMAGES] Images : 1477
[FOLDE

In [14]:
dataGeneration = ImageDataGenerator(validation_split=0.2,rescale=1./255,rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [15]:
trainGenerator = dataGeneration.flow_from_directory(
        '/content/raw/color/',
        target_size=(256,256),
        batch_size=64,
        class_mode='categorical',
        shuffle=True,
        subset='training')
print(trainGenerator)
classes = trainGenerator.class_indices

Found 43447 images belonging to 38 classes.


In [16]:
def ClassesLabel(classes):
    j=0
    for i in classes :
        f = open("Classes.csv","a+")
        f.write(str(j)+"@#"+i+"\n")
        j = j+1

In [17]:
ClassesLabel(classes)

In [18]:
 validGenerator = dataGeneration.flow_from_directory(
        '/content/raw/color/',
        target_size=(256,256),
        batch_size=64,
        class_mode='categorical',
        shuffle=True,
        subset='validation')

Found 10848 images belonging to 38 classes.


In [25]:
model = Sequential()

In [26]:
inputShape = (256,256,3)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
n_classes = 38
from keras.regularizers import l2

In [27]:
model.add(Conv2D(32, (3, 3), padding="same",name="conv_1",kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.20))
model.add(Conv2D(64, (3, 3),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.30))
model.add(Conv2D(128, (3, 3),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [28]:
 model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
activation_8 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 85, 85, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_9 (Activation)    (None, 85, 85, 64)       

In [29]:
 opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

[INFO] training network...


In [ ]:
 history = model.fit_generator(
    trainGenerator,steps_per_epoch = 43432//64,epochs = 6,validation_data = validGenerator,verbose=1)

Epoch 1/6
678/678 [==============================] - 1040s 2s/step - loss: 4.3465 - accuracy: 0.6425 - val_loss: 7.4666 - val_accuracy: 0.3535
Epoch 2/6
678/678 [==============================] - 1035s 2s/step - loss: 3.2129 - accuracy: 0.7966 - val_loss: 7.2728 - val_accuracy: 0.5601
Epoch 3/6
300/678 [============>.................] - ETA: 8:07 - loss: 2.7206 - accuracy: 0.8418

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()
 
plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
 print("[INFO] Calculating model accuracy")
scores = model.evaluate(validGenerator)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
 print("[INFO] Saving model...")
model.save("disease.h5")

In [ ]:
from keras.models import load_model
model2 = load_model("disease.h5")

In [ ]:
model2.summary()

In [ ]:
  opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model2.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
imagePath = listdir('/content/Test')

In [ ]:
for image in imagePath:
    print("[IMAGE] : "+str(image))
    path = '/content/Test/'+image
    img = cv2.imread(path)
    img = cv2.resize(img,(256,256))
 
    img = np.reshape(img,[1,256,256,3])
    img = img/255
    cl = model.predict_classes(img)
    print("[CLASS] : "+str(cl[0]))

In [ ]:
import pandas as pd